In [18]:
import os,glob
import time
import copy
import math

import numpy as np
import pandas as pd

import SimpleITK as sitk
import imageio

from file_io import load_image

def remove_bg(imgfile,bgcolor='black'):
    image = imageio.imread(imgfile).astype(np.uint8)
    print(image.shape)
    if len(image.shape)==2:
        bg = image==0
        print(len(np.nonzero(bg)[0]))
        array = 255*np.ones(list(image.shape)+[4]).astype(np.uint8)
        array[:,:,0] = image
        array[:,:,1] = image
        array[:,:,2] = image
        array[bg,3] = 0
        imageio.imsave(imgfile.replace('.png','_rmbg.png'),array)
        print(array.shape)
    return

In [36]:
brain_list = ['194069.v3draw','190891.v3draw','190827.v3draw','WHOLE_mouse_Brain_AZ10_SR3B_6_A_647.v3draw','A3_16um_final.v3draw','Ex_642_Em_680.v3draw']
axis_list = [2,1,1,0,0,0]
raw_list = []
raw_annotation_list = []
raw_removed_list = []
register_list = []
register_annotation = load_image('../assets/annotation_25.nrrd') .astype(np.uint16)
register_removed_list = []
for i,j in zip(brain_list,axis_list):
    print(i,j)
    original_file = '/PBshare/SEU-ALLEN/Users/ZhixiYun/data/registration/brainimg/CH2_Brains_DS_BE/' + i
    raw_file = './raw/' + i
    raw_list.append(raw_file)
    #os.system(f'cp {original_file} {raw_file}')
    
    original_annotation_file = '/PBshare/SEU-ALLEN/Users/ZhixiYun/data/registration/Inverse/' + i
    raw_annotation_file = './raw_annotation/' + i
    raw_annotation_list.append(raw_annotation_file)
    #os.system(f'cp {original_annotation_file} {raw_annatation_file}')
    
    raw_removed_file = './raw_removed/' + i[:-7] + '.tiff'
    raw_removed_list.append(raw_removed_file)
    raw = load_image(raw_file)[0].astype(np.uint8)
    raw_annotation = load_image(raw_annotation_file)[0].astype(np.uint16)
    raw[raw_annotation==0] = 0
    raw_removed = sitk.GetImageFromArray(raw)
    sitk.WriteImage(raw_removed, raw_removed_file)
    
    section = raw.max(axis=j)
    imgfile = raw_removed_file.replace('.tiff','.png')
    imageio.imsave(imgfile,section)
    remove_bg(imgfile)
    
    local_file = '/PBshare/SEU-ALLEN/Users/ZhixiYun/data/registration/brainimg/Local_noseg/' + i[:-7] + '/' + i[:-7] + '_local_registered_image.v3draw'
    register_file = './register/' + i
    register_list.append(register_file)
    os.system(f'cp {local_file} {register_file}')
 
    register_removed_file = './register_removed/' + i[:-7] + '.tiff'
    register_removed_list.append(register_removed_file)
    register = load_image(register_file)[0].astype(np.uint8)
    register[register_annotation==0] = 0
    register_removed = sitk.GetImageFromArray(register)
    sitk.WriteImage(register_removed, register_removed_file)    

    section = register.max(axis=1)
    imgfile = register_removed_file.replace('.tiff','.png')
    imageio.imsave(imgfile,section)
    remove_bg(imgfile)
    

190891.v3draw 1
(782, 503)
167930
(782, 503, 4)
(456, 528)
76067
(456, 528, 4)
